In [ ]:
#

In [1]:
 pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime

# Load the Excel file
file_path = '/Users/aswathy.n/Desktop/QuickRegressionTesting1.xlsx'
url_sheet_name = 'URLSheet'
data_sheet_name = 'DataSheet'

# Read the URL from the first sheet
df_url = pd.read_excel(file_path, sheet_name=0)
url_column = 'URL'
base_url = df_url.at[0, url_column]

# Read the input texts and expected responses from the specified sheet
df_data = pd.read_excel(file_path, sheet_name=data_sheet_name)
skuCol = 'SKU'
questionCol = 'Question'
expected_response_column = 'Expected Response'


In [2]:
print(pd.__version__)

2.2.2


In [55]:
# Load the Excel file
excel_file = pd.ExcelFile('/Users/aswathy.n/Desktop/QuickRegressionTesting5.xlsx')

# List all sheet names
sheet_names = excel_file.sheet_names
print("Sheet names:", sheet_names)

# Read the first sheet into a DataFrame
df_sheet1 = pd.read_excel(excel_file, sheet_name='URLSheet')
print("First sheet data:")
print(df_sheet1)
Magellen_AIWidget_QA=df_sheet1.loc[4,'URL']

# Read the third sheet into a DataFrame
df_sheet3 = pd.read_excel(excel_file, sheet_name='QA-MagellenCitations')
print("Third sheet data:")
print(df_sheet3.head())

Sheet names: ['URLSheet', 'Deployment_CountryList', 'MandatoryQs', 'MandatoryQ', 'Prod-PDPAIWidget', 'QA-PDPAIWidget', 'QA3-FSHS', 'QA-MagellenCitations', 'QA_Trials', 'Prod_trials', 'Sheet2']
First sheet data:
                                                 URL
0  https://www.thermofisher.com/order/catalog/pro...
1  https://www.qa.thermofisher.com/order/catalog/...
2  https://qa3.fishersci.com/us/en/customer-help-...
3  https://fishersci.com/us/en/customer-help-supp...
4  https://www.qa.thermofisher.com/antibody/produ...
Third sheet data:
                                            Question      SKU  \
0  does this come in liquid form? what does this ...  MA1-110   
1                                       How to login  MA1-110   

                                     Expected Answer  
0  For the most accurate and up-to-date product i...  
1  Sorry, I can only assist with inquiries relate...  


In [56]:
Magellen_AIWidget_QA

'https://www.qa.thermofisher.com/antibody/product/Nestin-Antibody-clone-10C2-Monoclonal/MA1-110'

In [57]:
# Initialize the web driver (make sure to have the appropriate driver for your browser)
driver = webdriver.Chrome() #executable_path='/Users/aswathy.n/Documents/chromedriver')
driver.maximize_window()

In [58]:

now=datetime.datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S")
print ("current date time is "+ str(formatted_date_time))

current date time is 2025-12-24 14:18:45


#This is for hard coded run
# Initialize lists to store results
matches = []
mismatches = []

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0][1]

# Function to modify the URL
def modify_url(base_url, new_value):
    modified_url = re.sub(r'/product/[^?]+', f'/product/{new_value}', base_url)
    
    modified_url = re.sub(r'srp-[^&]+', f'srp-{new_value}', modified_url)
    modified_url="https://www.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061"
    return modified_url

# Initialize the previous URL modifier value
previous_url_modifier = None

# Open the web portal with the modified URL
modified_url="https://www.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061"
driver.get(modified_url)


time.sleep(10)

cookie_accept=driver.find_element(By.ID, 'truste-consent-button')
cookie_accept.click()
time.sleep(15)


In [59]:
# Initialize lists to store results
matches = []
mismatches = []

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0][1]

# Function to modify the URL
def modify_url(base_url, new_value):
    modified_url = re.sub(r'/product/[^?]+', f'/product/{new_value}', base_url)
    
    modified_url = re.sub(r'srp-[^&]+', f'srp-{new_value}', modified_url)
   # modified_url="https://www.thermofisher.com/order/catalog/product/NP00061?SID=srch-srp-NP00061"
    print (modified_url, new_value)
    return modified_url

# Initialize the previous URL modifier value
previous_url_modifier = None

def set_cookie(driver, name, value):
    driver.add_cookie({'name': name, 'value': value})

# Function to delete the cookie
def delete_cookie(driver, name):
    driver.delete_cookie(name)



In [60]:
# Open the web portal with the modified URL
base_url=Magellen_AIWidget_QA
country_code='US'
driver.get(base_url)

time.sleep(10)

cookie_accept=driver.find_element(By.ID, 'truste-consent-button')
cookie_accept.click()
time.sleep(15)


# Delete the existing cookie CK_ISO_CODE
delete_cookie(driver, 'CK_ISO_CODE')
#Setting new value
set_cookie(driver, 'CK_ISO_CODE', country_code)

delete_cookie(driver, 'ts_magellan_genai')
#Setting new value
set_cookie(driver, 'ts_magellan_genai', 'true')


driver.refresh()
time.sleep(10)



In [61]:
#To be included later!!!
# Iterate over the rows in the DataFrame
for index, row in df_sheet3.iterrows():
    SKU = row['SKU']
    Question = row['Question']
    expected_response = row['Expected Answer']
    print ("Question, expected_response, SKU")
    print (Question,expected_response, SKU)

    # Modify the URL only if the URL modifier value changes
    if SKU != previous_url_modifier:
        modified_url = modify_url(base_url, SKU)
        previous_url_modifier = SKU
        driver.get(modified_url)
        print (modified_url, base_url)
        time.sleep(10)
        # Click on the specific tab before entering the input text
        
        
        
    
    #  /html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div
       # segment_tab.click()     #/html/body/div[10]/div/div[2]/div[2]/core-navigationbar/core-navigationbar-item[2]//div                   
        #/html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div
        # /html/body/div[9]/div/div[2]/div[2]/core-navigationbar/core-navigationbar-item[2]//div/a 
        # Wait for the tab to load (adjust the sleep time as needed) #/html/body/div[9]/div/div[3]/div[2]/core-navigationbar/core-navigationbar-item[2]//div/a
        
    else:
        driver.get(modified_url)
        time.sleep(10)
        print ("inside else")


    refSection = driver.find_element(By.XPATH, '/html/body/div[8]/div/div/div/div/div[2]/div/div[2]/div[1]/div[1]/div/div[1]/div[4]/div[3]/a') #
    refSection.click()
    time.sleep(10)

    preText=driver.find_element(By.CSS_SELECTOR, '#prompt-container-inline-with-scroll > span:nth-child(1)')
    preText_text=preText.text
    print (preText_text)

    pubSummaryLink=driver.find_element(By.CSS_SELECTOR, '#prompt-container-inline-with-scroll > span:nth-child(1) > div > div > ul > li:nth-child(2) > a')
    pubsummaryName= pubSummaryLink.text
    pubSummaryLink.click()

    time.sleep(10)
    pubSummaryLinkSummary=driver.find_element(By.CSS_SELECTOR, '#prompt-container-inline-with-scroll > span:nth-child(1) > div.message.bot')
    pubSummaryLinkSummaryText=pubSummaryLinkSummary.text
    print (pubSummaryLinkSummaryText)

    
    
    # Find the input field and enter the text
    textArea=driver.find_element(By.ID, 'core-prompt__textfield-input')
    textArea.click()
    textArea.send_keys("Hello")

    # Wait for the response to be generated (2 minutes)
    time.sleep(50)

    # Find the response field and get the text
    #response_field = driver.find_element(By.CLASS_NAME, 'input-search hydrated')  # Adjust the locator as needed
    #generated_response = response_field.text

   # response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[2]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div/div[2]') # '//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div/div[2]/div/div[1]/div/div/div/div')  # Adjust the locator as needed
   #  response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div')
   # response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div/div[2]/div/div[1]/div/div/div')
   # response_field = driver.find_element(By.CSS_SELECTOR, '#search-component-wrapper-ssw > core-card > div > div.input-search-wrapper-ssw > core-search')
    response_field = driver.find_element(By.XPATH,'/html/body/div[9]/div/div[3]/div[3]/div[2]/smart-search-web-component/div/core-card/div/div[2]/div/div[2]')
    generated_response = response_field.text       
    print("generated_response")
    print(generated_response)
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div/div[2]/div/div[1]/div/div/div/div
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div
                                                    #//*[@id="search-component-wrapper-ssw"]/core-card/div/div[2]/div[3]/div[2]/div/div[1]/div
    # Calculate cosine similarity
    similarity = calculate_cosine_similarity(generated_response, expected_response)

    # Compare the generated response with the expected response using cosine similarity
    if similarity >= 0.60:  # Threshold for considering a match (adjust as needed)
        matches.append((SKU,Question ,generated_response, expected_response, similarity))
    else:
        mismatches.append((SKU,Question ,generated_response, expected_response, similarity))

# Close the web driver
#driver.quit()

print(matches)
print(mismatches)

Question, expected_response, SKU
does this come in liquid form? what does this contain? For the most accurate and up-to-date product information including whether it comes in liquid form and its contents, please refer to the details provided on this product page. MA1-110
https://www.qa.thermofisher.com/antibody/product/MA1-110 MA1-110
https://www.qa.thermofisher.com/antibody/product/MA1-110 https://www.qa.thermofisher.com/antibody/product/Nestin-Antibody-clone-10C2-Monoclonal/MA1-110
The Thermo Fisher Scientific product MA1-110 is widely utilized across various experimental applications, demonstrating its versatility in scientific research. In immunocytochemistry and immunohistochemistry, it is frequently applied to study cellular processes in models such as C. elegans, house mice, and human samples, focusing on neural differentiation, disease pathophysiology, and therapeutic strategies. Its use in flow cytometry and Western Blot further highlights its role in analyzing cellular marker

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="core-prompt__textfield-input"]"}
  (Session info: chrome=143.0.7499.170); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x00000001050c0b2c cxxbridge1$str$ptr + 3033596
1   chromedriver                        0x00000001050b8b30 cxxbridge1$str$ptr + 3000832
2   chromedriver                        0x0000000104bb2b0c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x0000000104bf99c8 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 364688
4   chromedriver                        0x0000000104c3a968 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 630832
5   chromedriver                        0x0000000104bee16c _RNvCsgXDX2mvAJAg_

In [ ]:
matches

In [ ]:
mismatches

In [ ]:

now=datetime.datetime.now()
formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S")
print ("current date time is "+ str(formatted_date_time))

In [ ]:
matches

In [ ]:
mismatches

In [ ]:
matchingResults="matches_report_QAPDP_"+country_code+"_"+str(now.strftime("%Y-%m-%d %H_%M_%S"))+".csv"
mismatchingResults="mismatches_report_QAPDP_"+country_code+"_"+str(now.strftime("%Y-%m-%d %H_%M_%S"))+".csv"

In [ ]:
matchingResults, mismatchingResults

In [ ]:
import datetime
# Generate the report
matches_report = pd.DataFrame(matches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
matches_report.to_csv(matchingResults, index=False)

mismatches_report = pd.DataFrame(mismatches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
mismatches_report.to_csv(mismatchingResults, index=False)

print("Report generated successfully.")

# Generate the report
matches_report = pd.DataFrame(matches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
matches_report.to_excel('matches_report_QA.xlsx', index=False)

mismatches_report = pd.DataFrame(mismatches, columns=["SKU","Question", 'Generated Response', 'Expected Response', 'Cosine Similarity'])
mismatches_report.to_excel('mismatches_report_QA.xlsx', index=False)

print("Report generated successfully.")